In [3]:
%cd test

/notebooks/test


# 1. Download "LLaMA-7B"

In [ ]:
from huggingface_hub import snapshot_download

# Download the repository
repo_id = "nyanko7/LLaMA-7B"
local_dir = "./LLaMA-7B"  # Directory to save the files
snapshot_download(repo_id, local_dir=local_dir)

print(f"All files downloaded to: {local_dir}")

# 2. Download "COCO2014"
- Train2014 only

In [ ]:
!mkdir coco2014
!wget http://images.cocodataset.org/zips/train2014.zip
!unzip train2014.zip -d coco2014/

# 3. Download "alpaca_gpt4_data.json"

In [6]:
import requests

# Raw URL of the file
url = "https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json"
# Local filename to save the file
filename = "LLaMA-Adapter_replicated/llama_adapter_v2_multimodal7b/alpaca_gpt4_data.json"

# Send a GET request to the URL
response = requests.get(url, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Open the file in write mode and save the content
    with open(filename, "w", encoding="utf-8") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk.decode("utf-8"))
    print(f"File downloaded successfully and saved as {filename}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


File downloaded successfully and saved as alpaca_gpt4_data.json


# 4. Download "llava_instruct_150k.json"

In [ ]:
import requests

file_url = "https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json"
output_file = "llava_instruct_150k.json"
response = requests.get(file_url)
if response.status_code == 200:  # Check if the request was successful
    with open(output_file, "wb") as f:
        f.write(response.content)
    print(f"File downloaded and saved as '{output_file}'.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

# 5. Update "llava_instruct_150k.json" to reassign "image" value

In [ ]:
import json
import os

# Paths
json_path = "llava_instruct_150k.json"
image_folder = "coco2014/train2014"
output_json_path = "LLaMA-Adapter_replicated/llama_adapter_v2_multimodal7b/updated_llava_instruct_150k.json"

# Load the JSON file
with open(json_path, 'r') as file:
    data = json.load(file)

# Update the image paths
for entry in data:
    # Original image name
    image_name = entry["image"]
    # Format the image path to match the required structure
    entry["image"] = os.path.join(image_folder, f"COCO_train2014_{image_name.split('.')[0]}.jpg")

# Save the updated JSON to a new file
with open(output_json_path, 'w') as file:
    json.dump(data, file, indent=2)

print(f"Updated JSON saved to: {output_json_path}")

# 6. Download the pre-trained 7B model
- Pre-trained on Image-Text-V1, which is a concatenation of LAION400M, COYO, MMC4, SBU, Conceptual Captions, and COCO

In [ ]:
!wget -O 7B-pretrained.pth "https://huggingface.co/Cxxs/ImageBind-LLM/resolve/main/7B-pretrained.pth"

# 7. Install requirements

In [2]:
%cd LLaMA-Adapter_replicated/llama_adapter_v2_multimodal7b
!pip install -r requirements.txt
%cd ../..

# 8. Training

In [ ]:
!python -u LLaMA-Adapter_replicated/llama_adapter_v2_multimodal7b/main_finetune.py --data_config LLaMA-Adapter_replicated/llama_adapter_v2_multimodal7b/finetune_data_config.yaml \
 --batch_size 4 \
 --epochs 4 --warmup_epochs 1 --blr 10e-4 --weight_decay 0.02 \
 --llama_path LLaMA-7B \
 --output_dir Output \
 --pretrained_path 7B-pretrained.pth

# 9. Resume training (Optional; if the training was stopped)

In [ ]:
!python -u LLaMA-Adapter_replicated/llama_adapter_v2_multimodal7b/main_finetune.py --data_config LLaMA-Adapter_replicated/llama_adapter_v2_multimodal7b/finetune_data_config.yaml \
 --batch_size 4 \
 --epochs 4 --warmup_epochs 1 --blr 10e-4 --weight_decay 0.02 \
 --llama_path LLaMA-7B \
 --output_dir Output 
 --pretrained_path /path/to/checkpoint.pth \
 --start_epoch [epoch]